<a href="https://colab.research.google.com/github/reedyjw1/CattleCounterColab/blob/master/cattle_counter_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Upload Files
Here you can upload the "hive.csv" that contains all of the image links and bounding box annotations.

In [ ]:
from google.colab import files
files.upload()

## Dependencies

In this section, Tensorflow's object detection API will be installed and setup. Becuase of issues converting Tensorflow 2.x models to TFLite, Tensorflow 1.x will be used for model training.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf


In [ ]:
!pip install tqdm
!pip install colorama
!pip install wget
!pip install lvis
!pip install tensorboard

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=6cea3777c89cfdeab4552a09cce5a2e54c72323c194f2a66ace83a20897a24b4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


Clone tensorflow to use the object detection API


In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 50720, done.
remote: Total 50720 (delta 0), reused 0 (delta 0), pack-reused 50720
Receiving objects: 100% (50720/50720), 568.66 MiB | 48.31 MiB/s, done.
Resolving deltas: 100% (34547/34547), done.


Install dependencies needed for object detection

In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/u

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 51 not upgraded.
Need to get 1,615 kB of archives.
After this operation, 8,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

Convert the proto files to python files

In [ ]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.


/content/models/research/object_detection/protos
object_detection/protos/flexible_grid_anchor_generator.proto: File not found.
object_detection/protos/grid_anchor_generator.proto: File not found.
object_detection/protos/multiscale_anchor_generator.proto: File not found.
object_detection/protos/ssd_anchor_generator.proto: File not found.
anchor_generator.proto: Import "object_detection/protos/flexible_grid_anchor_generator.proto" was not found or had errors.
anchor_generator.proto: Import "object_detection/protos/grid_anchor_generator.proto" was not found or had errors.
anchor_generator.proto: Import "object_detection/protos/multiscale_anchor_generator.proto" was not found or had errors.
anchor_generator.proto: Import "object_detection/protos/ssd_anchor_generator.proto" was not found or had errors.
anchor_generator.proto:14:5: "GridAnchorGenerator" is not defined.
anchor_generator.proto:15:5: "SsdAnchorGenerator" is not defined.
anchor_generator.proto:16:5: "MultiscaleAnchorGenerator" i

Add object detection program to the python path

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

In [ ]:
!echo $PYTHONPATH

/tensorflow-1.15.2/python3.6:/env/python:/content/models/research/:/content/models/research/slim:/content/models/research/:/content/models/research/slim:/content/models/research/:/content/models/research/slim


Install tf_slim dependency

In [ ]:
!pip3 install tf_slim

     |████████████████████████████████| 358kB 5.2MB/s 


Run the model_builder_test to make sure we installed everything correctly

In [ ]:

%cd /content/models/research/object_detection/builders/
!python model_builder_test.py

/content/models/research/object_detection/builders


## Generate the TFRecords
In this section, the images from the "hive.csv" and the annotations are converted into TFRecords for Tensorflow to use when training the model.

Get or create your tfrecords for training and testing. 

In [ ]:
%cd /content/models/research

/content/models/research


In [ ]:
import csv
import requests
import sys
import getopt
from getopt import GetoptError
import traceback
import re
import os
import json
from collections import namedtuple
import requests
import tqdm
import tensorflow as tf
import random
from object_detection.utils import dataset_util

In [ ]:
%cd /content/
!mkdir /content/cattle_counter_data
%cd /content/cattle_counter_data

/content
/content/cattle_counter_data


In [ ]:
# Make sure you run this in google colab
# You'll also have to clone tensorflow models for the object detection utils
# git clone https://github.com/tensorflow/models

Boxes = namedtuple('Boxes', [
    'x_minimums',
    'y_minimums',
    'x_maximums',
    'y_maximums',
    'label_indexes',
    'label_names'
])

bad_images = {
    # "thumb3034.jpg",
    # "thumb4089.jpg",
    # "thumb8117.jpg",
    # "thumb8123.jpg",
    # "thumb8121.jpg",
    # "thumb8127.jpg",
    # "thumb8131.jpg",
    # "thumb8129.jpg",
    # "thumb8126.jpg",
    # "thumb8125.jpg",
    # "thumb1075.jpg",
    # "thumb4090.jpg",
    # "thumb4092.jpg",
    # "thumb4093.jpg",
    # "thumb4094.jpg",
    # "thumb4085.jpg",
    # "thumb4095.jpg",
    # "thumb4091.jpg",
    # "thumb4096.jpg",
    # "thumb5014.jpg",
    # "thumb8125.jpg",
    # "thumb8126.jpg"
}

In [ ]:
def get_num_records(input_file):
    num_records = 0
    with open(input_file, 'rt') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        num_records = len(csv_file.readlines()) - 1
    return num_records


def create_tfrecord_from_csv(input_file, output_file, split=0.9):
    writer = tf.io.TFRecordWriter(output_file)
    validation_record_file = re.sub(
        "\.tfrecord",
        "_val.tfrecord",
        output_file
    )
    validation_writter = tf.io.TFRecordWriter(validation_record_file)
    with open(input_file, 'rt') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        num_records = get_num_records(input_file)
        validation_index = int(num_records * split)
        count = 0
        csv_data = []
        for row in tqdm.tqdm(csv_reader, total=num_records):
            if (row["original_filename"] not in bad_images):
                csv_data.append({
                    "status": row["status"],
                    "download_url": row["download_url"],
                    "media_height": row["media_height"],
                    "media_width": row["media_width"]
                })
        # shuffle records before saving them to the tf record
        random.shuffle(csv_data)
        for row in tqdm.tqdm(csv_data):
            data = json.loads(row["status"])
            if (len(data) != 0):
                boxes = get_boxes(data)
                file_name = "cattle{count}.jpg".format(count=count).encode('utf8')
                image_data = requests.get(row["download_url"])
                tf_example = tf.train.Example(features=tf.train.Features(feature={
                    'image/height': dataset_util.int64_feature(int(row["media_height"])),
                    'image/width': dataset_util.int64_feature(int(row["media_width"])),
                    'image/filename': dataset_util.bytes_feature(file_name),
                    'image/source_id': dataset_util.bytes_feature(file_name),
                    'image/encoded': dataset_util.bytes_feature(image_data.content),
                    'image/format': dataset_util.bytes_feature(b'jpg'),
                    'image/object/bbox/xmin': dataset_util.float_list_feature(boxes.x_minimums),
                    'image/object/bbox/xmax': dataset_util.float_list_feature(boxes.x_maximums),
                    'image/object/bbox/ymin': dataset_util.float_list_feature(boxes.y_minimums),
                    'image/object/bbox/ymax': dataset_util.float_list_feature(boxes.y_maximums),
                    'image/object/class/text': dataset_util.bytes_list_feature(boxes.label_names),
                    'image/object/class/label': dataset_util.int64_list_feature(boxes.label_indexes),
                }))

                if (count >= validation_index):
                    validation_writter.write(tf_example.SerializeToString())
                else:
                    writer.write(tf_example.SerializeToString())
                count += 1
        writer.close()
        validation_writter.close()


def get_boxes(data):
    boxes = Boxes([], [], [], [], [], [])
    for detection in data:
        boxes.x_minimums.append(detection["p1"]["x"])
        boxes.y_minimums.append(detection["p1"]["y"])
        boxes.x_maximums.append(detection["p2"]["x"])
        boxes.y_maximums.append(detection["p2"]["y"])
        boxes.label_indexes.append(1)
        boxes.label_names.append("cattle".encode('utf8'))
    return boxes

#input_file="/content/hive.csv"
input_file = '/content/hive.csv'
validation_split=0.9
create_tfrecord_from_csv(input_file, "cattle.tfrecord", validation_split)

100%|██████████| 5762/5762 [13:01<00:00,  7.38it/s]


In [ ]:
def create_pbtxt_file(labels):
  with open('cattle.pbtxt', 'wt') as label_file:
    i = 1
    for label in labels:
      item_str = "\tid: {id}\n\tname: '{name}'\n".format(id=i, name=label)
      item_str = "item{\n" + item_str + "}"
      if (i != len(labels)):
        item_str += ",\n"
      else:
        item_str += "\n"
      label_file.write(
          item_str
      )
      i +=1

create_pbtxt_file(['cattle'])

In [ ]:

train_record_fname = '/content/cattle_counter_data/cattle.tfrecord'
test_record_fname = '/content/cattle_counter_data/cattle_val.tfrecord'
label_map_pbtxt_fname = '/content/cattle_counter_data/cattle.pbtxt'

In [ ]:
%cd /content/

/content


## Train the Model
SSD Mobilenet V3 is the model that will be used to train the cattle counter. In this section, the training is prepared and started. The batch size and the number of steps can be changed.

In [ ]:
model_name = 'ssd_mobilenet_v3_small_coco_2020_01_14'
download_config = 'http://download.tensorflow.org/models/object_detection/{model_name}.tar.gz'.format(model_name=model_name)
!wget {download_config}
!tar -zxvf { model_name + '.tar.gz'}

--2021-01-21 16:08:40--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.103.128, 2607:f8b0:4001:c13::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.103.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27341125 (26M) [application/gzip]
Saving to: ‘ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz’

ssd_mobilenet_v3_sm 100%[===================>]  26.07M   125MB/s    in 0.2s    

2021-01-21 16:08:41 (125 MB/s) - ‘ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz’ saved [27341125/27341125]

ssd_mobilenet_v3_small_coco_2020_01_14/
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.index
ssd_mobilenet_v3_small_coco_2020_01_14/frozen_inference_graph.pb
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.data-00000-of-00001
ssd_mobilenet_v3_small_coco_2020_01_14/model.ckpt.meta
ssd_mobilenet_v3_small_coco_2020_01_14/pipeline.c

In [ ]:
#prepare
pipeline_fname = '/content/' + model_name + '/pipeline.config'
fine_tune_checkpoint = '/content/' + model_name + '/model.ckpt'
label_map_pbtxt_fname = '/content/cattle_counter_data/cattle.pbtxt'
num_classes = 1
batch_size = 64
num_steps = 5

In [ ]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd {'/content/' + model_name + '/'}
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        'input_path: "PATH_TO_BE_CONFIGURED\/.*_train.*"', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        'input_path: "PATH_TO_BE_CONFIGURED\/.*_val.*"', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
     #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
    
    s = re.sub(
        'learning_rate_base: \d\.\d*', 'learning_rate_base: {}'.format(0.001), s)
    s = re.sub(
        'warmup_learning_rate: \d\.\d*', 'warmup_learning_rate: {}'.format(0.001), s)
    """
    s = re.sub(
        'width: \d+', 'width: 640', s
    )

    s = re.sub('height: \d+', 'height: 640', s)
  """
    f.write(s)



/content/ssd_mobilenet_v3_small_coco_2020_01_14
writing custom configuration file


In [ ]:
%cat {pipeline_fname}

# SSDLite with Mobilenet v3 large feature extractor.
# Trained on COCO14, initialized from scratch.
# TPU-compatible.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: tru

In [ ]:
!cp /content/models/research/object_detection/legacy/train.py /content/train.py
%cd /content/
!python /content/train.py --logtostderr --train_dir=training/ --pipeline_config_path={pipeline_fname}

In [ ]:
!cp /content/models/research/object_detection/legacy/eval.py /content/eval.py
%cd /content/
!python eval.py --logtostderr --pipeline_config_path={pipeline_fname} --checkpoint_dir=training/ --eval_dir=eval/

## Exporting Inference Graph

In [ ]:
%cd /content/models/research/object_detection
!python3 export_inference_graph.py --input_type image_tensor --pipeline_config_path {pipeline_fname} --trained_checkpoint_prefix /content/training/model.ckpt-2000 --output_directory /content/trained_model

## Calculate Accuracy

In [ ]:
%cd /content/
!git clone https://github.com/reedyjw1/ConfusionMatrix.git

In [ ]:
!cp /content/models/research/object_detection/inference/infer_detections.py /content/ConfusionMatrix/infer_detections.py
%cd /content/ConfusionMatrix/
!python infer_detections.py --input_tfrecord_paths={train_record_fname} --output_tfrecord_path=train_detections.record --inference_graph=/content/trained_model/frozen_inference_graph.pb

In [ ]:
!python confusion_matrix_edited.py --detections_record=train_detections.record --label_map=/content/cattle_counter_data/cattle.pbtxt --output_path=confusion_matrix.csv

In [ ]:
def calculateAccuracy(csvFile):
    originalRows = []

    with open(csvFile, newline='') as file:
        reader = csv.reader(file, delimiter=",", quotechar='"')
        next(reader)
        for row in reader:
            accuracy = (float(row[0]) + float(row[3]))/(float(row[0]) + float(row[1]) + float(row[2]) + float(row[3])) * 100
            print("Accuracy: " + "{:.2f}".format(accuracy) + "%")
        
calculateAccuracy("confusion_matrix.csv")

Accuracy: 11.11%


## Testing the Model





In [ ]:
%cd /content/models/research/object_detection

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = '/content/trained_model'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/cattle_counter_data/cattle.pbtxt'
NUM_CLASSES = 1


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)



PATH_TO_TEST_IMAGES_DIR = '/content/cattle_counter_data/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'test.jpg') ]
IMAGE_SIZE = (320, 320)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
    
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict




for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  print(output_dict)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1)
  display(Image.fromarray(image_np))
  



## Export TFLite model and Label File

In [ ]:
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=/content/trained_model/pipeline.config \
--trained_checkpoint_prefix=/content/trained_model/model.ckpt \
--output_directory=/content/tflite \
--add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0119 16:52:36.927084 140594120533888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0119 16:52:38.770811 140594120533888 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0119 16:52:38.843403 140594120533888 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0119 16:52:38.915388 140594120533888 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [ ]:
!sudo apt install curl gnupg
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor > bazel.gpg
!sudo mv bazel.gpg /etc/apt/trusted.gpg.d/
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel
!sudo apt install bazel-3.1.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.12).
gnupg is already the newest version (2.2.4-1ubuntu1.3).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8
Get:1 https://storage.googleapis.com/bazel-apt stable InRelease [2,256 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Releas

In [ ]:
%cd /content/
!git clone --recurse-submodules https://github.com/tensorflow/tensorflow.git

/content
Cloning into 'tensorflow'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1072782 (delta 37), reused 42 (delta 21), pack-reused 1072704
Receiving objects: 100% (1072782/1072782), 645.73 MiB | 33.62 MiB/s, done.
Resolving deltas: 100% (873843/873843), done.
Checking out files: 100% (24196/24196), done.


In [ ]:
%cd /content/tensorflow/
!bazel run -c opt tensorflow/lite/toco:toco -- \
--input_file=/content/tflite/tflite_graph.pb \
--output_file=/content/tflite/detect.tflite \
--input_shapes=1,320,320,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
--inference_type=FLOAT \
--mean_values=128 \
--std_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

Streaming output truncated to the last 5000 lines.
    Compiling tensorflow/core/grappler/graph_topology_view.cc; 7s local
    Compiling tensorflow/core/grappler/utils/topological_sort.cc; 5s local
[2,187 / 2,776] 4 actions running
    Compiling tensorflow/core/grappler/mutable_graph_view.cc; 12s local
    Compiling tensorflow/core/grappler/graph_topology_view.cc; 8s local
    Compiling tensorflow/core/grappler/utils/topological_sort.cc; 6s local
[2,188 / 2,776] 4 actions, 3 running
    Compiling tensorflow/core/grappler/mutable_graph_view.cc; 13s local
    Compiling tensorflow/core/grappler/utils/topological_sort.cc; 7s local
    Compiling tensorflow/core/grappler/utils/traversal.cc; 5s local
[2,188 / 2,776] 4 actions running
    Compiling tensorflow/core/grappler/mutable_graph_view.cc; 14s local
    Compiling tensorflow/core/grappler/utils/topological_sort.cc; 7s local
    Compiling tensorflow/core/grappler/utils/traversal.cc; 5s local
[2,188 / 2,776] 4 actions running
    Compiling 

In [ ]:
interpreter = tf.lite.Interpreter(model_path='/content/tflite/detect.tflite')
interpreter.allocate_tensors()

print(interpreter.get_input_details())

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(interpreter.get_output_details())

from keras.preprocessing import image
import numpy as np
import tensorflow as tf
# predicting images
path = '/content/cattle_counter_data/test.jpg'
img = image.load_img(path, target_size=(320, 320))
x = image.img_to_array(img, dtype=np.float32) 
x = np.expand_dims(x, axis=0)

interpreter.set_tensor(input_index,x)
interpreter.invoke()
prediction = interpreter.get_tensor(output_index)
print(prediction)

## Download Files

In [ ]:
f = open("/content/label.txt", "w")
f.write('cattle')
f.close()

In [ ]:
%cd /content/

saved_dir = '/content/cattle_counter_files/'
tflite_path = '/content/tflite/detect.tflite'
label_txt = '/content/label.txt'
model_training_dir = '/content/training/'
inference_graph_dir = '/content/trained_model/'

!mkdir {saved_dir}

!cp {tflite_path} {saved_dir}
!cp {label_txt} {saved_dir}
!cp {model_training_dir} {saved_dir} -r
!cp {inference_graph_dir} {saved_dir} -r

/content


In [ ]:
!zip -r /content/file.zip /content/cattle_counter_files

  adding: content/cattle_counter_files/ (stored 0%)
  adding: content/cattle_counter_files/trained_model/ (stored 0%)
  adding: content/cattle_counter_files/trained_model/model.ckpt.index (deflated 66%)
  adding: content/cattle_counter_files/trained_model/saved_model/ (stored 0%)
  adding: content/cattle_counter_files/trained_model/saved_model/saved_model.pb (deflated 19%)
  adding: content/cattle_counter_files/trained_model/saved_model/variables/ (stored 0%)
  adding: content/cattle_counter_files/trained_model/frozen_inference_graph.pb (deflated 17%)
  adding: content/cattle_counter_files/trained_model/model.ckpt.meta (deflated 93%)
  adding: content/cattle_counter_files/trained_model/pipeline.config (deflated 69%)
  adding: content/cattle_counter_files/trained_model/checkpoint (deflated 42%)
  adding: content/cattle_counter_files/trained_model/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: content/cattle_counter_files/label.txt (stored 0%)
  adding: content/cattle_counter_fil

In [ ]:
files.download('file.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>